# Imports

In [1]:
# ============================================================================
# 04_results.py
# Loads and visualizes results from 03_modeling.py
# Includes model comparison, ROC/PR curves, confusion matrices, feature importance, and DCA
# ============================================================================

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve, auc
from pathlib import Path

# Load Models and Results

In [3]:
# ============================================================================
# 1. LOAD MODELS AND RESULTS
# ============================================================================
print("="*80)
print("LOAD MODELS AND RESULTS")
print("="*80)

# Festgelegter Pfad zu deinem models-Ordner
models_dir = Path(r"C:\Users\Eyyub\Desktop\StackFuel\PortfolioProjekt\DPP-Stackfuel-Data-Science-Projekt\models")

# Lade die gespeicherten Modell-Ergebnisse
model_results = joblib.load(models_dir / "model_results.pkl")

# Lade die einzelnen Modelle (nach Name)
models = {}
for model_name in model_results.keys():
    model_path = models_dir / f"{model_name.replace(' ', '_')}_model.pkl"
    models[model_name] = joblib.load(model_path)

# Lade die Test-Labels
target_test = joblib.load(models_dir / "target_test.pkl")

print(f"Loaded models: {list(models.keys())}")
print(f"Test labels shape: {target_test.shape}")


LOAD MODELS AND RESULTS
Loaded models: ['Random Forest', 'XGBoost', 'Logistic Regression', 'Neural Network', 'SVM', 'K-Nearest Neighbors', 'Decision Tree', 'Naive Bayes']
Test labels shape: (45895,)


# Model Comparison Table

In [4]:
# ============================================================================
# 2. MODEL COMPARISON TABLE
# ============================================================================
print("\n" + "="*80)
print("MODEL COMPARISON TABLE")
print("="*80)

comparison_data = []
for name, res in model_results.items():
    comparison_data.append({
        "Model": name,
        "AUC": res['auc'],
        "Precision": res['precision'],
        "Recall": res['recall'],
        "F1": res['f1'],
        "Accuracy": res['accuracy'],
        "Best Threshold": res.get('best_threshold', 0.5)
    })

df_comparison = pd.DataFrame(comparison_data).sort_values("F1", ascending=False)
print(df_comparison.to_string(index=False))



MODEL COMPARISON TABLE
              Model      AUC  Precision   Recall       F1  Accuracy  Best Threshold
            XGBoost 0.815416   0.370106 0.642399 0.469639  0.778102        0.260960
Logistic Regression 0.808352   0.378504 0.602080 0.464804  0.787951        0.634490
                SVM 0.808368   0.369580 0.624163 0.464261  0.779693        0.618986
        Naive Bayes 0.762289   0.311205 0.684570 0.427891  0.720035        0.950891
     Neural Network 0.769980   0.324482 0.609346 0.423465  0.746247        0.422415
      Random Forest 0.763384   0.323888 0.597663 0.420109  0.747663        0.270000
K-Nearest Neighbors 0.715296   0.278787 0.629862 0.386502  0.694193        0.600000
      Decision Tree 0.592259   0.287119 0.336943 0.310042  0.770650        1.000000
